In [1]:
!pip install tensorflow=='2.6.0'

In [2]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import six
import time

import imageio
import glob
import scipy
import numpy as np
import cv2
import csv
import scipy.misc
import pandas as pd
import lxml.etree as ET


import pathlib
import hashlib

from PIL import Image
from six import BytesIO
from IPython.display import display
%matplotlib inline

In [3]:
os.chdir('/content/drive/MyDrive/wobot_final')

In [4]:
if 'models' in pathlib.Path.cwd().parts:
  while 'models' in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2922, done.
remote: Counting objects: 100% (2922/2922), done.
remote: Compressing objects: 100% (2454/2454), done.
remote: Total 2922 (delta 741), reused 1285 (delta 430), pack-reused 0
Receiving objects: 100% (2922/2922), 32.98 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (741/741), done.
Checking out files: 100% (2671/2671), done.


In [5]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/drive/My Drive/wobot_final/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1665514 sha256=17327ddffbbc3bcc661db1e6ba134d35cab615b96fa847bcef8ac293e9340eb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-dgb516rs/wheels/88/dc/fa/e1176786248533d404d7a54ad9cc2caacb32e3f3a9ea0ae690
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=9f7aa3c3bb6066df0fe124fe9bf2ded211d833ea479d6d740b801310270db860
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=eaf7be642cd53f8f6843efca09cf6f575925f4484d9c559c7be1e7164801c592
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78546 sha256=eea5557be06370a8233f87

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
os.chdir('/content/drive/MyDrive/wobot_final/models/research/object_detection')

In [7]:
train_record_path = 'data/train.record'
test_record_path = 'data/test.record'
labelmap_path = 'data/tf_label_map.pbtxt'

In [8]:
batch_size = 16
num_steps = 8000
num_eval_steps = 1000

In [9]:
fine_tune_checkpoint = 'efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'
base_config_path = 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

In [10]:
import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(3), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [11]:
%cat model_config.config

 # SSD with EfficientNet-b0 + BiFPN feature extractor,
# shared box predictor and focal loss (a.k.a EfficientDet-d0).
# See EfficientDet, Tan et al, https://arxiv.org/abs/1911.09070
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from an EfficientNet-b0 checkpoint.
#
# Train on TPU-8

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 3
    add_background_class: false
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      m

In [12]:
model_dir = 'training/'
pipeline_config_path = 'model_config.config'
output_directory = '/content/drive/MyDrive/wobot_final/models/research/object_detection/inference_graph'

In [13]:
!python /content/drive/MyDrive/wobot_final/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

INFO:tensorflow:Step 5400 per-step time 2.330s
I0923 12:57:37.668171 140356333823872 model_lib_v2.py:700] Step 5400 per-step time 2.330s
INFO:tensorflow:{'Loss/classification_loss': 0.17304341,
 'Loss/localization_loss': 0.08950291,
 'Loss/regularization_loss': 0.03562736,
 'Loss/total_loss': 0.2981737,
 'learning_rate': 0.079981245}
I0923 12:57:37.668586 140356333823872 model_lib_v2.py:701] {'Loss/classification_loss': 0.17304341,
 'Loss/localization_loss': 0.08950291,
 'Loss/regularization_loss': 0.03562736,
 'Loss/total_loss': 0.2981737,
 'learning_rate': 0.079981245}
INFO:tensorflow:Step 5500 per-step time 2.346s
I0923 13:01:32.253006 140356333823872 model_lib_v2.py:700] Step 5500 per-step time 2.346s
INFO:tensorflow:{'Loss/classification_loss': 0.21883161,
 'Loss/localization_loss': 0.15568613,
 'Loss/regularization_loss': 0.03570034,
 'Loss/total_loss': 0.41021806,
 'learning_rate': 0.07997993}
I0923 13:01:32.253389 140356333823872 model_lib_v2.py:701] {'Loss/classification_loss'

In [17]:
!python /content/drive/MyDrive/wobot_final/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

2021-09-23 14:40:33.500057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 14:40:33.816894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 14:40:33.817874: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 14:40:33.835993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 14:40:33.836825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S